In [2]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import xarray as xr
import matplotlib.colors as pltc
import geopandas
import dataretrieval.nwis as nwis
import contextily as cx

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466


/opt/miniconda3/lib/python3.9/site-packages/dataretrieval/nadp.py:44: UserWarning: GDAL not installed. Some functions will not work.
  warnings.warn('GDAL not installed. Some functions will not work.')


#### Load NP sites from Drying Regimes

In [3]:
pnwNP = pd.read_csv("../data/pnwNP_Info.csv")

#### Cross-reference with CIG provided shapefile to get stream segment

In [5]:
shp = geopandas.read_file("../data/VIC_UW/shapefiles/columbia_seg.shp")
shp = shp.loc[shp['POI_ID'].isin(pnwNP["comid"])]
seg_ids = np.array(shp['seg_id'])

array([ 715,  774,  842,  866,  867,  868,  885,  909,  965, 1095, 1328,
       1338, 1565, 1913, 1928, 2005, 2008, 3372, 4252, 4557, 4865, 5164,
       5217, 5359, 5364, 5610, 5633])

#### Get VIC data for corresponding stream segments and write to new netCDF.

In [ ]:
## Open modeled datasets
data = xr.open_mfdataset('../data/VIC_UW/vic_historical_first_route_all.nc')

## Extract data at segment locations.
data = data.where(data['reachID'].isin(seg_ids),drop=True)

# # sort seg_ids in ascending order (not necessary)
# data['seg'] = sorted(seg_ids)

# # make new output directory
# Path(f'/home/disk/rocinante/rawrgers/Data/Data_Requests/Rupp/nwfsc_data/{hydro}/projections/first_route/{model}/{scenario}/').mkdir(parents=True, exist_ok=True)


# # # save the reachID (i.e., seg_ids) and IRFroutedRunoff (i.e., streamflow) to new netcdf file
# gage = '13152500'
# name = '../data/VIC_UW/' + gage
name = "../data/pnwNP_Gages_VIC.nc"

# out = data[['reachID','IRFroutedRunoff']].to_netcdf(name)

#### Get PRMS data for corresponding stream segments and write to new netCDF.

In [12]:
## Open modeled datasets
data = xr.open_mfdataset('../data/VIC_UW/prms_historical_first_route_all.nc')

## Extract data at segment locations.
data = data.where(data['reachID'].isin(seg_ids),drop=True)

# # sort seg_ids in ascending order (not necessary)
# data['seg'] = sorted(seg_ids)

# # make new output directory
# Path(f'/home/disk/rocinante/rawrgers/Data/Data_Requests/Rupp/nwfsc_data/{hydro}/projections/first_route/{model}/{scenario}/').mkdir(parents=True, exist_ok=True)


# # # save the reachID (i.e., seg_ids) and IRFroutedRunoff (i.e., streamflow) to new netcdf file
# gage = '13152500'
# name = '../data/VIC_UW/' + gage
name = "../data/pnwNP_Gages_PRMS.nc"

# out = data[['reachID','IRFroutedRunoff']].to_netcdf(name)

## Example for Matt

In [20]:
seg_ids = [909]

## Open modeled datasets
data = xr.open_mfdataset('../data/VIC_UW/vic_historical_first_route_all.nc')

## Extract data at segment locations.
data = data.where(data['reachID'].isin(seg_ids),drop=True)

dat = data[['reachID','IRFroutedRunoff']]
comb = dat.to_dataframe()
comb

,,reachID,IRFroutedRunoff
time,seg,,
1950-01-01,0,909.0,1.769737
1950-01-02,0,909.0,3.594224
1950-01-03,0,909.0,4.431946
1950-01-04,0,909.0,4.676565
1950-01-05,0,909.0,4.722897
...,...,...,...
2011-12-27,0,909.0,6.929360
2011-12-28,0,909.0,7.278639
2011-12-29,0,909.0,9.860347


In [93]:
maladVIC = pd.read_csv('../data/UW_StreamflowOutput/historical_livneh_VIC_P1-WOODGOOD-streamflow-1.0.csv',comment='#')
maladVIC['date'] = pd.to_datetime(maladVIC['date'])
maladVIC.columns = ['time','streamflowVIC']
maladVIC['streamflowVIC'] = maladVIC['streamflowVIC'] * cfs_2_cms
maladVIC

,time,streamflowVIC
0,1950-01-01,1.552698
1,1950-01-02,2.379748
2,1950-01-03,2.383910
3,1950-01-04,2.380371
4,1950-01-05,2.376831
...,...,...
22640,2011-12-27,2.257646
22641,2011-12-28,2.446745
22642,2011-12-29,7.097136
22643,2011-12-30,9.468021


## Scratch space